In [ ]:
from dotenv import load_dotenv
load_dotenv('.env')
from typing import Optional
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from momentfm import MOMENTPipeline
import torch
import numpy as np
import torch
import torch.cuda.amp
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import OneCycleLR
from tqdm import tqdm
from momentfm.utils.utils import control_randomness


/home/zhupengtian/miniconda3/envs/zql_moment/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zhupengtian/miniconda3/envs/zql_moment/lib/python3.11/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:

fl = 48
batch_size = 24
torch.cuda.empty_cache()


In [ ]:
# 数据处理部分
class ECLDataset:
    def __init__(
        self,
        forecast_horizon: Optional[int] = fl,
        data_path :str = '/home/zhupengtian/zhangqingliang/moment/data/electricity.csv' ,
        data_split: str = "train",
        data_stride_len: int = 1,
        task_name: str = "forecasting",
        random_seed: int = 42,
    ):

        self.seq_len = 512
        self.forecast_horizon = forecast_horizon
        self.full_file_path_and_name = data_path
        self.data_split = data_split
        self.data_stride_len = data_stride_len
        self.task_name = task_name
        self.random_seed = random_seed

        # Read data
        self._read_data()
        
    def _get_borders(self, train_pct: float = 0.1, val_pct: float = 0.05, test_pct: float = 0.85):
        # 计算数据集总长度
        total_length = self.length_timeseries_original

        # 计算各个数据集的结束位置
        n_train = int(total_length * train_pct)
        n_val = int(total_length * val_pct)
        n_test = int(total_length * test_pct)

        # 确保数据集划分之和不超过总数据集长度
        assert n_train + n_val + n_test <= total_length, "Total split exceeds the length of the dataset."

        # 计算边界
        train_end = n_train
        val_end = n_train + n_val
        test_start = val_end
        test_end = test_start + n_test

        # 划分数据集的切片
        train = slice(0, train_end)
        val = slice(train_end, val_end)
        test = slice(test_start, test_end)

        return train,test

    def _read_data(self):
        self.scaler = StandardScaler()
        df = pd.read_csv(self.full_file_path_and_name)
        self.length_timeseries_original = df.shape[0]
        self.n_channels = df.shape[1] - 1

        df.drop(columns=["date"], inplace=True)
        df = df.infer_objects(copy=False).interpolate(method="cubic")

        data_splits = self._get_borders()

        train_data = df[data_splits[0]]
        self.scaler.fit(train_data.values)
        df = self.scaler.transform(df.values)

        if self.data_split == "train":
            self.data = df[data_splits[0], :]
        elif self.data_split == "test":
            self.data = df[data_splits[1], :]

        self.length_timeseries = self.data.shape[0]

    def __getitem__(self, index):
        seq_start = self.data_stride_len * index
        seq_end = seq_start + self.seq_len
        input_mask = np.ones(self.seq_len)

        if self.task_name == "forecasting":
            pred_end = seq_end + self.forecast_horizon

            if pred_end > self.length_timeseries:
                pred_end = self.length_timeseries
                seq_end = seq_end - self.forecast_horizon
                seq_start = seq_end - self.seq_len

            timeseries = self.data[seq_start:seq_end, :].T
            forecast = self.data[seq_end:pred_end, :].T

            return timeseries, forecast, input_mask

        elif self.task_name == "imputation":
            if seq_end > self.length_timeseries:
                seq_end = self.length_timeseries
                seq_end = seq_end - self.seq_len

            timeseries = self.data[seq_start:seq_end, :].T

            return timeseries, input_mask

    def __len__(self):
        if self.task_name == "imputation":
            return (self.length_timeseries - self.seq_len) // self.data_stride_len + 1
        elif self.task_name == "forecasting":
            return (
                self.length_timeseries - self.seq_len - self.forecast_horizon
            ) // self.data_stride_len + 1
        


In [ ]:
# 评估函数
# 计算 MSE (Mean Squared Error)
def mse(trues, preds):
    return np.mean((trues - preds) ** 2)

# 计算 MAE (Mean Absolute Error)
def mae(trues, preds):
    return np.mean(np.abs(trues - preds))

# 计算 SMAPE (Symmetric Mean Absolute Percentage Error)
def smape(trues, preds):
    numerator = np.abs(trues - preds)
    denominator = (np.abs(trues) + np.abs(preds)) / 2
    return 200 * np.mean(numerator / denominator)

# 计算 MAPE (Mean Absolute Percentage Error)
def mape(trues, preds):
    return np.mean(np.abs((trues - preds) / trues)) * 100

# 计算 MASE (Mean Absolute Scaled Error)
def mase(trues, preds, historical_values):
    # 计算基准模型（例如：上一时刻值）误差
    naive_error = np.abs(trues[1:] - trues[:-1])
    forecast_error = np.abs(trues - preds)
    return np.mean(forecast_error[1:]) / np.mean(naive_error)


In [ ]:

# 导入模型
model = MOMENTPipeline.from_pretrained(
    "/home/zhupengtian/zhangqingliang/models/MOMENT-1-large", 
    model_kwargs={
        'task_name': 'forecasting',
        'forecast_horizon': fl,
        'head_dropout': 0.1,
        'weight_decay': 0,
        'freeze_encoder': True, # Freeze the patch embedding layer
        'freeze_embedder': True, # Freeze the transformer encoder
        'freeze_head': False, # The linear forecasting head must be trained
    },
    local_files_only=True,  # Whether or not to only look at local files (i.e., do not try to download the model).
)
model.init()


In [ ]:

# 加载数据集，设置训练参数
# Set random seeds for PyTorch, Numpy etc.
control_randomness(seed=13) 

# Load data
train_dataset = ECLDataset(data_split="train", random_seed=13, forecast_horizon=fl)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = ECLDataset(data_split="test", random_seed=13, forecast_horizon=fl)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 训练周期数
cur_epoch = 0
max_epoch = 1

# Move the model to the GPU
model = model.to(device)

# Move the loss function to the GPU
criterion = criterion.to(device)

# Enable mixed precision training
scaler = torch.cuda.amp.GradScaler()

# Create a OneCycleLR scheduler
max_lr = 1e-4
total_steps = len(train_loader) * max_epoch
scheduler = OneCycleLR(optimizer, max_lr=max_lr, total_steps=total_steps, pct_start=0.3)

# Gradient clipping value
max_norm = 5.0


In [ ]:

# 微调模型（预测头）
while cur_epoch < max_epoch:
    losses = []
    for timeseries, forecast, input_mask in tqdm(train_loader, total=len(train_loader)):
        # Move the data to the GPU
        timeseries = timeseries.float().to(device)
        input_mask = input_mask.to(device)
        forecast = forecast.float().to(device)

        with torch.cuda.amp.autocast():
            output = model(x_enc=timeseries, input_mask=input_mask)
        
        loss = criterion(output.forecast, forecast)

        # Scales the loss for mixed precision training
        scaler.scale(loss).backward()

        # Clip gradients
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)

        losses.append(loss.item())

    losses = np.array(losses)
    average_loss = np.average(losses)
    print(f"Epoch {cur_epoch}: Train loss: {average_loss:.3f}")

    # Step the learning rate scheduler
    scheduler.step()
    cur_epoch += 1


In [ ]:

# Evaluate the model on the test split
trues, preds, histories, losses = [], [], [], []
model.eval()
with torch.no_grad():
    for timeseries, forecast, input_mask in tqdm(test_loader, total=len(test_loader)):
    # Move the data to the GPU
        timeseries = timeseries.float().to(device)
        input_mask = input_mask.to(device)
        forecast = forecast.float().to(device)

        with torch.cuda.amp.autocast():
            output = model(x_enc=timeseries, input_mask=input_mask)
        
        loss = criterion(output.forecast, forecast)                
        losses.append(loss.item())

        trues.append(forecast.detach().cpu().numpy())
        preds.append(output.forecast.detach().cpu().numpy())
        histories.append(timeseries.detach().cpu().numpy())

losses = np.array(losses)
average_loss = np.average(losses)
model.train()

trues = np.concatenate(trues, axis=0)
preds = np.concatenate(preds, axis=0)
histories = np.concatenate(histories, axis=0)

# 计算各项指标
mse = mse(trues, preds)
mae = mae(trues, preds)
smape = smape(trues, preds)
mape = mape(trues, preds)
historical_values = trues[:-1]  # 用上一时刻的值作为基准
mase = mase(trues, preds, historical_values)
print(f"MSE: {mse:.6f}")
print(f"MAE: {mae:.6f}")
print(f"SMAPE: {smape:.6f}")
print(f"MAPE: {mape:.6f}")
print(f"MASE: {mase:.6f}")